### IMPORTING NECESSARY PACKAGES  
  
The project will require:  
 - NumPy and Pandas for Data Frames  
 - tabula to be able to extract data from PDF file  
 - re in order to use regular expression for data cleaning

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import tabula
import re

### 1. ILLICIT CIGARETTES  
 - Illicit cigarretes was extracted manually from different PROJECT SUN done by KPMG, collected thorugh the years  
 - Volumes declared were in billion stick

In [2]:
# Importing data
illicit27 = pd.read_csv(r"C:\Users\Gordan\Desktop\Data Analyst\Projects\EU27 Tobacco Analysis\Illicit EU27.csv")  

# For further analysis it was important to assign category type
illicit27.insert(1, 'Category', 'Illicit Cigarettes', True)  
  
# Rename column for filtering purposes    
illicit27.rename(columns={'EU27': 'Country'}, inplace=True)  

### 2. RETAIL CIGARETTES  
 - The data has been imported from the folowing source: https://taxation-customs.ec.europa.eu/system/files/2023-12/tobacco_products_releases-consumption.pdf  
 - The cigarette volumes declared were in 1000 sticks

In [3]:
cig27 = pd.DataFrame(tabula.read_pdf(r"https://taxation-customs.ec.europa.eu/system/files/2023-12/tobacco_products_releases-consumption.pdf", pages ='all')[0])

#### 2.1. RESIZING AND CATEGORISING

In [4]:
# assigning category type  
cig27.insert(1, 'Category', 'Cigarettes', True)  
  
# limiting years from 2014 onwards    
cig27.drop(cig27.iloc[:, 2:14],axis = 1, inplace= True)  
  
# excluding 2022 from analysis    
cig27 = cig27.iloc[:, :-1]  
  
# excluding UK and Total to limit analysis to EU27    
cig27 = cig27.iloc[:-2,:]  
  
# Standardizing first column for filtering purposes
cig27['EU'] = illicit27['Country']
cig27 = cig27.rename(columns={'EU': 'Country'})

#### 2.2. CLEANING AND STANDARDIZING VALUES

In [5]:
# Remove N/A data
cig27 = cig27.replace(to_replace='N/A', value=0, regex=True)  
  
# Remove "," in order to avoid decimal sign and thousandt delimiter
cig27 = cig27.replace(',', value='', regex=True)  
  
# Remove data entry errors using Regular Expressions
cig27.iloc[26,:] = cig27.iloc[26,:].replace(to_replace='(\r[0-9]+$)', value = '', regex = True)  
  
# Transform DataFrame values into billion and make the data type float 
cig27.iloc[:, 2:23] = cig27.iloc[:, 2:23].astype(float)/1000000

C:\Users\Gordan\AppData\Local\Temp\ipykernel_22980\3751302888.py:11: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  cig27.iloc[:, 2:23] = cig27.iloc[:, 2:23].astype(float)/1000000


### 3. SMOKING TOBACCO  
 - The data has been imported from the folowing source: https://taxation-customs.ec.europa.eu/system/files/2023-12/tobacco_products_releases-consumption.pdf  
 - The cigarette volumes declared were in kg  
 
 - One stick rolled cigarette in European Union takes arround 0.75 grams of smoking tobacco  
 - The study that mentions this value can be found on: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4127802/  
 - With values mentioned above the amount of sticks from one kg of smoking tobacco is 1,333.333 The formula is:  
 1kg/0.00075kg(0.75g)= 1,333.333

In [6]:
tob27 = pd.DataFrame(tabula.read_pdf(r"https://taxation-customs.ec.europa.eu/system/files/2023-12/tobacco_products_releases-consumption.pdf", pages ='all')[1])

#### 3.1. RESIZING AND CATEGORISING

In [7]:
# assigning category type  
tob27.insert(1, 'Category', 'Tobacco', True)  
  
# limiting years from 2014 onwards    
tob27.drop(tob27.iloc[:, 2:14],axis = 1, inplace= True)  
  
# excluding 2022 from analysis    
tob27 = tob27.iloc[:, :-1]  
  
# excluding UK and Total to limit analysis to EU27    
tob27 = tob27.iloc[:-2,:]  
  
# Standardizing first scolumn for filtering purposes
tob27['EU'] = illicit27['Country']
tob27 = tob27.rename(columns={'EU': 'Country'})

#### 3.2. CLEANING AND STANDARDIZING VALUES

In [8]:
# Remove N/A data
tob27 = tob27.replace(to_replace='N/A', value=0, regex=True)  
  
# Remove "," in order to avoid decimal sign and thousandt delimiter
tob27 = tob27.replace(',', value='', regex=True)  
  
# Transform DataFrame values into data type float  
tob27.iloc[:, 2:23] = tob27.iloc[:, 2:23].astype(float)

# Use the formula to tranform Kilograms to sticks
tob27.iloc[:, 2:23] = tob27.iloc[:, 2:23]*1333.3/1000000000

C:\Users\Gordan\AppData\Local\Temp\ipykernel_22980\2877360665.py:8: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  tob27.iloc[:, 2:23] = tob27.iloc[:, 2:23].astype(float)


### 4. MERGING AND TRANSFORMING DATAFRAMES

In [9]:
# Creating unified DataFrame for necessary analysis
Tobacco_Total = pd.concat([cig27, illicit27, tob27], sort=False, axis=0)  
  
# Transposing data for easier analysis with SQL and Power BI
Tobacco_Total = pd.melt(Tobacco_Total, id_vars=['Country', 'Category'], var_name='Year', value_name='Volumes')  
 

In [10]:
Tobacco_Total

,Country,Category,Year,Volumes
0,Austria,Cigarettes,2014,12.902235
1,Belgium,Cigarettes,2014,10.820948
2,Bulgaria,Cigarettes,2014,11.343246
3,Croatia,Cigarettes,2014,5.654912
4,Cyprus,Cigarettes,2014,1.347888
...,...,...,...,...
643,Romania,Tobacco,2021,0.112197
644,Slovakia,Tobacco,2021,0.430939
645,Slovenia,Tobacco,2021,0.209877
646,Spain,Tobacco,2021,8.291793


In [11]:
Tobacco_Total.to_csv(r'C:\Users\Gordan\Desktop\Data Analyst\Projects\EU27 Tobacco Analysis\EU27_Total_Tobacco_Volumes_source.csv')